In [186]:
import numpy as np

class DATA():
    def __init__(self, matrix, label):
        self.matrix = matrix
        self.label = label    

In [187]:
trn_data = {}
prior = {}

# Implement all training data
with open("optdigits-orig_train.txt") as f:
    matrix = []
    for line in f:
        if len(line) != 3:
            matrix.append(line.rstrip())
        else:
            class_idx = int(line.rstrip())
            observation = DATA(matrix, line.rstrip())
            if class_idx not in trn_data:
                trn_data[class_idx] = [observation]
            else:
                trn_data[class_idx] = trn_data[class_idx] + [observation]
            matrix = []

# Prior Probability for each number
total = 0
for i in range(10):
    total += len(trn_data[i])
for i in range(10):
    prior[i] = len(trn_data[i]) / total
prior


tst_data = {}
with open("optdigits-orig_test.txt") as f:
    matrix = []
    for line in f:
        if len(line) != 3:
            matrix.append(line.rstrip())
        else:
            class_idx = int(line.rstrip())
            observation = DATA(matrix, line.rstrip())
            if class_idx not in trn_data:
                tst_data[class_idx] = [observation]
            else:
                tst_data[class_idx] = trn_data[class_idx] + [observation]
            matrix = []

In [188]:
# P(f_ij = 0 | class)
# P(f_ij = 1 | class)
feature_dict_1 = {}
feature_dict_0 = {}
for i in range(10):
    feature_dict_1[i] = np.zeros((32, 32))
    feature_dict_0[i] = np.zeros((32, 32))

for class_idx in range(10):
    digit_data = trn_data[class_idx]
    feature_prob_1 = feature_dict_1[class_idx] 
    feature_prob_0 = feature_dict_0[class_idx] 
    for i in range(32):
        for j in range(32): 
            f = 0
            num = 0
            for digit in digit_data:
                if digit.matrix[i][j] == "1":
                    f += 1
                num += 1
            # Laplace Smoothing
            feature_prob_1[i][j] = (f+1)/(num+1)
            feature_prob_0[i][j] = (num - f + 1)/(num+1)

In [183]:
def err_rate(actual, predict):
    return sum(actual != predict) / len(predict)

In [185]:
# testing for a single test data
trial = tst_data[0][0]
for row in trial.matrix:
    print(row)

prob = np.zeros((9,))

for class_idx in range(9):
    feature_class_1 = feature_dict_1[class_idx]
    feature_class_0 = feature_dict_0[class_idx]
    log_sum = np.log(prior[class_idx])
    for i in range(32):
        for j in range(32):
            if trial.matrix[i][j] == "0":
                log_sum += np.log(feature_class_0[i][j])
            else:
                log_sum += np.log(feature_class_1[i][j])
                
    prob[class_idx] = log_sum

print(" ")
print("Predicted label:", np.argmax(prob))

00000000000001111000000000000000
00000000000011111110000000000000
00000000001111111111000000000000
00000001111111111111100000000000
00000001111111011111100000000000
00000011111110000011110000000000
00000011111110000000111000000000
00000011111110000000111100000000
00000011111110000000011100000000
00000011111110000000011100000000
00000011111100000000011110000000
00000011111100000000001110000000
00000011111100000000001110000000
00000001111110000000000111000000
00000001111110000000000111000000
00000001111110000000000111000000
00000001111110000000000111000000
00000011111110000000001111000000
00000011110110000000001111000000
00000011110000000000011110000000
00000001111000000000001111000000
00000001111000000000011111000000
00000001111000000000111110000000
00000001111000000001111100000000
00000000111000000111111000000000
00000000111100011111110000000000
00000000111111111111110000000000
00000000011111111111110000000000
00000000011111111111100000000000
00000000001111111110000000000000
0000000000

In [63]:
# Visualization
for data in trn_data[0]:
    print("Label:", data.label)
    for row in data.matrix:
        print(row)
    print(" ")

Label:  0
00000000000001111000000000000000
00000000000011111110000000000000
00000000001111111111000000000000
00000001111111111111100000000000
00000001111111011111100000000000
00000011111110000011110000000000
00000011111110000000111000000000
00000011111110000000111100000000
00000011111110000000011100000000
00000011111110000000011100000000
00000011111100000000011110000000
00000011111100000000001110000000
00000011111100000000001110000000
00000001111110000000000111000000
00000001111110000000000111000000
00000001111110000000000111000000
00000001111110000000000111000000
00000011111110000000001111000000
00000011110110000000001111000000
00000011110000000000011110000000
00000001111000000000001111000000
00000001111000000000011111000000
00000001111000000000111110000000
00000001111000000001111100000000
00000000111000000111111000000000
00000000111100011111110000000000
00000000111111111111110000000000
00000000011111111111110000000000
00000000011111111111100000000000
00000000001111111110000000000000


00000011111100000000011111000000
00000001111100000000011111000000
00000001111100000000011111000000
00000001111100000000011111000000
00000001111000000000011111000000
00000001111000000000011111000000
00000011111000000000011111000000
00000001111100000000011111000000
00000001111100000000011111000000
00000001111100000000111111000000
00000001111110000000111111000000
00000000111110000000111110000000
00000000111110000011111100000000
00000000011111111111111100000000
00000000011111111111111000000000
00000000001111111111111000000000
00000000001111111111110000000000
00000000000001111111110000000000
00000000000000111111100000000000
 
Label:  0
00000000000001100000000000000000
00000000001111111000000000000000
00000000001111111111100000000000
00000000011111111111110000000000
00000000011111111111111000000000
00000000111111011111111100000000
00000000111100001111111100000000
00000000111100000111111110000000
00000001111000000110111110000000
00000001111000000100011110000000
0000000111100000000001111100000

00000000111110000000011111000000
00000000111110000000011111000000
00000000111110000000011111000000
00000001111100000000111100000000
00000001111110000000111110000000
00000001111110000000111100000000
00000001111111000000111100000000
00000000111111111111111100000000
00000000011111111111111100000000
00000000011111111111111100000000
00000000011111111111111100000000
00000000001111111111111000000000
00000000001111111111100000000000
00000000000000011111000000000000
 
Label:  0
00000000000001111000000000000000
00000000000011111110000000000000
00000000000111111111000000000000
00000000001111111111110000000000
00000000111111111111110000000000
00000000111111111111111000000000
00000000111111111111111000000000
00000001111111000001111000000000
00000001111111000000111100000000
00000011111111000000111110000000
00000001111111110000001111000000
00000001111111110000000111000000
00000001111111110000000111000000
00000001111011110000000111000000
00000001110001110000000011000000
0000000111000000000000011100000

00000000000111111111111100000000
00000000000011111111110000000000
00000000000000111110000000000000
 
Label:  0
00000000000000011000000000000000
00000000000111111110000000000000
00000000001111111111100000000000
00000000001111111111110000000000
00000000011111111111111000000000
00000000011111100011111000000000
00000000111110000001111000000000
00000000111110000001111100000000
00000000111110000000111110000000
00000001111110000000111110000000
00000001111110000000011111000000
00000001111110000000001111000000
00000001111110000000001111100000
00000001111100000000001111000000
00000001111000000000001111000000
00000001111000000000001111000000
00000001111000000000000111000000
00000000111100000000000111000000
00000000111100000000000111000000
00000000111100000000000111000000
00000001111000000000011110000000
00000001111000000000011110000000
00000000111000000000011110000000
00000000111110000011111110000000
00000000111110001111111100000000
00000000111111111111111000000000
0000000001111111111111100000000

0.0